In [1]:
import findspark
import os
import gc
#findspark.init() 
SPARK_HOME='/opt/cloudera/parcels/CDH/lib/spark'
findspark.init(SPARK_HOME)

import time
import math
import copy
import csv
import json
import os
import codecs
import subprocess
import numpy as np
from pyspark import SQLContext
from pyspark.sql import Row
from pyspark.sql import functions as F
from pyspark.sql.functions import size, array_union,flatten,array_sort,coalesce,broadcast,collect_list, collect_set, udf, array_remove, log, lit, first, col, array, sort_array,split, explode, desc, asc, row_number,isnan, when, count
from pyspark.sql.types import *
import rtree
from pyspark.sql import Window
import geofeather
from pyspark.storagelevel import StorageLevel

from pyspark.sql import SparkSession
from pyspark import StorageLevel
import geopandas as gpd
import pandas as pd
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, LongType, FloatType, ArrayType, MapType
from shapely.geometry import Point
from shapely.geometry import Polygon

from sedona.register import SedonaRegistrator
from sedona.core.SpatialRDD import SpatialRDD, PointRDD, CircleRDD, PolygonRDD, LineStringRDD
from sedona.core.enums import FileDataSplitter
from sedona.utils.adapter import Adapter
from sedona.core.spatialOperator import KNNQuery
from sedona.core.spatialOperator import JoinQuery
from sedona.core.spatialOperator import JoinQueryRaw
from sedona.core.spatialOperator import RangeQuery
from sedona.core.spatialOperator import RangeQueryRaw
from sedona.core.formatMapper.shapefileParser import ShapefileReader
from sedona.core.formatMapper import WkbReader
from sedona.core.formatMapper import WktReader
from sedona.core.formatMapper import GeoJsonReader
from sedona.sql.types import GeometryType
from sedona.core.enums import GridType
from sedona.core.SpatialRDD import RectangleRDD
from sedona.core.enums import IndexType
from sedona.core.geom.envelope import Envelope
from sedona.utils import SedonaKryoRegistrator, KryoSerializer

os.environ['PYSPARK_PYTHON'] = "./environment/bin/python"
#os.environ['PYSPARK_PYTHON'] = "/home/qiany/.conda/envs/py37/bin/python"
os.environ['YARN_CONF_DIR'] = "/opt/cloudera/parcels/CDH/lib/spark/conf/yarn-conf"

In [2]:
tin_file = input("Here is a programe to extract boundary relations, please input the absolute or relative path to your .ts file:")

# get the directory, basename of the input file
print("\n********************")
tin_directory = os.path.dirname(tin_file)
print("tin_directory: ", tin_directory)

directory_type = input("Is the data stored in hdfs(0) or Tri_data(1) or Tetra_data (2):") or "2"

if directory_type == '0':
    directory = 'hdfs_data'
elif directory_type == '1':
    directory = 'Tri_data'
else:
    directory = 'Tetra_data'
    
tin_basename = os.path.basename(tin_file) # input_vertices_2.off
print("tin_basename: ", tin_basename)

tin_filename = os.path.splitext(tin_basename)[0] # input_vertices_2
print("tin_filename: ", tin_filename)

tin_extension = os.path.splitext(tin_basename)[1] # .off
print("tin_extension: ", tin_extension)

print("\n********************")
print("This is a TIN file in \"%s\" format" % tin_extension)

filtra = 'yes'

# allocate the number of executors, the number of cores per executor, and the amount of memory per executor
Num_executor = '64'
Num_core_per_executor = '5'
Memory_executor = '64g'
MemoryOverhead_executor = '8g'

# allocate the number of cores for the driver node
Num_core_per_driver = '5'
Memory_driver = '64g'
MemoryOverhead_driver = '32g'

# the default number of shuffle partitions
Num_shuffle_partitions = input("spark.sql.shuffle.partitions:")


********************
tin_directory:  /home/qiany/yuehui/pyspark/Tetra_mesh/data
tin_basename:  Lander_big.ts
tin_filename:  Lander_big
tin_extension:  .ts

********************
This is a TIN file in ".ts" format


In [3]:
'''
spark.executor.cores: # Number of concurrent tasks an executor can run, euqals to the number of cores to use on each executor
spark.executor.instances: # Number of executors for the spark application
spark.executor.memory: # Amount of memory to use for each executor that runs the task
spark.executor.memoryOverhead:
spark.driver.cores: # Number of cores to use for the driver process; the default number is 1
spark.driver.memory: # Amount of memory to use for the driver
spark.driver.maxResultSize: to define the maximum limit of the total size of the serialized result that a driver can store for each Spark collect action
spark.default.parallelism: # Default number of partitions in RDDs returned by transformations like join, reduceByKey, and parallelize when not set by user. It can be set as spark.executor.instances * spark.executor.cores * 2
spark.sql.shuffle.partitions: determine how many partitions are used when data is shuffled between nodes, e.g., joins or aggregations. usually 1~5 times of executor.instances * executor.cores
spark.memory.storageFraction: determines the fraction of the heap space that is allocated to caching RDDs and DataFrames in memory.
spark.kryoserializer.buffer.max: determine the maximum of data that can be serialized at once; this must be larger than any object we attempt to serialize
spark.rpc.message.maxSize: # Maximum message size (in MiB) to allow in "control plane" communication; generally only applies to map output size information sent between executors and the driver. To communicate between the nodes, Spark uses a protocol called RPC (Remote Procedure Call), which sends messages back and forth. The spark.rpc.message.maxSize parameter limits how big these messages can be. 
spark.sql.broadcastTimeout: Spark will wait for this amount of time before giving up on broadcasting a table. Broadcasting can take a long time if the table is large or if there is a shuffle operation before it.
spark.sql.autoBroadcastJoinThreshold: Spark will broadcast a table to all worker nodes when performing a join if its size is less than this value; -1 means disabling broadcasting
'''

date = time.strftime("%m,%d,%Y")
date_name = date.split(',')[0] + date.split(',')[1] + date.split(',')[2]

hour = time.strftime("%H,%M")
hour_name = hour.split(',')[0] + hour.split(',')[1]

Topo = input("What kind of connectivity relation do you want to retrieve?")

spark_app_name = "TopoRela_Adjacent_TetraMesh_" + tin_filename + '_' + date_name + '_' + hour_name + '_' + Topo
print("spark_app_name:", spark_app_name)

spark = SparkSession \
.builder \
.appName(spark_app_name) \
.master('yarn') \
.config("spark.serializer", KryoSerializer.getName) \
.config('spark.jars','sedona-core-2.4_2.11-1.0.0-incubating.jar,sedona-sql-2.4_2.11-1.0.0-incubating.jar,sedona-python-adapter-2.4_2.11-1.0.0-incubating.jar,sedona-viz-2.4_2.11-1.0.0-incubating.jar,geotools-wrapper-geotools-24.0.jar,graphframes-0.8.0-spark2.4-s_2.11.jar') \
.config('spark.executor.cores', Num_core_per_executor) \
.config('spark.executor.instances', Num_executor) \
.config('spark.executor.memory', Memory_executor) \
.config('spark.executor.memoryOverhead', MemoryOverhead_executor) \
.config('spark.driver.cores', Num_core_per_driver) \
.config('spark.driver.memory', Memory_driver) \
.config('spark.driver.memoryOverhead', MemoryOverhead_driver) \
.config('spark.driver.maxResultSize', '0') \
.config('spark.dynamicAllocation.enabled', 'false') \
.config('spark.network.timeout', '10000001s') \
.config('spark.executor.heartbeatInterval', '10000000s') \
.config('spark.sql.shuffle.partitions', Num_shuffle_partitions) \
.config("spark.default.parallelism", '400') \
.config("spark.kryoserializer.buffer.max", "1024mb") \
.config('spark.rpc.message.maxSize', '256') \
.config("spark.sql.broadcastTimeout", "36000") \
.config("spark.sql.autoBroadcastJoinThreshold", "-1") \
.config("spark.sql.objectHashAggregate.sortBased.fallbackThreshold", "-1") \
.config('spark.yarn.dist.archives', '/local/data/yuehui/py37.tar.gz#environment') \
.config("spark.python.profile", "true") \
.config("spark.eventLog.enabled", "true") \
.config("spark.eventLog.logStageExecutorMetrics", "true") \
.getOrCreate()

What kind of connectivity relation do you want to retrieve? TT_M1


spark_app_name: TopoRela_Adjacent_TetraMesh_Lander_big_06152024_1343_TT_M1


### read input files

In [4]:
# read_tetra_order() is a function used to read tetrahedron from a csv file
def read_tetra_order(hdfs_tetra_origin):
    '''
    this function has two input parameters.
    filtra: 'yes' or 'no', yes means that the input csv file is ordered by default
    directory: a string denoting the directory to a tetrahedra file
    tin_filename: a string denoting the file name of a tetrahedra extension, e.g., 827_monviso
    '''
        
    schema_tetra_origin = StructType([ \
        StructField("tetra_order",IntegerType(),True), \
        StructField("r1",IntegerType(),True), \
        StructField("r2",IntegerType(),True), \
        StructField("r3",IntegerType(),True), \
        StructField("r4",IntegerType(),True), \
        StructField("r1_ele",FloatType(),True), \
        StructField("r2_ele",FloatType(),True), \
        StructField("r3_ele",FloatType(),True), \
        StructField("r4_ele",FloatType(),True) \
      ])

    df_tetra_origin = spark.read.format("csv") \
          .option("header", False) \
          .schema(schema_tetra_origin)\
          .load(hdfs_tetra_origin)
        
    return df_tetra_origin


# read tetrahedra
hdfs_tetra_origin = directory + "/" + tin_filename + '_filtra_tetra_sort.csv'

df_tetra_order = read_tetra_order(hdfs_tetra_origin)
df_tetra_order.printSchema()

Number of partitions for df_tetra_origin: 4959
root
 |-- r1: integer (nullable = true)
 |-- r2: integer (nullable = true)
 |-- r3: integer (nullable = true)
 |-- r4: integer (nullable = true)
 |-- r1_ele: float (nullable = true)
 |-- r2_ele: float (nullable = true)
 |-- r3_ele: float (nullable = true)
 |-- r4_ele: float (nullable = true)



# VV relation

### Method 1: pure global

In [5]:
# function to get VV using groupby
def get_VV(df_tetra_order):
    df_VV_init_1 = df_tetra_order.select("r1", "r2", "r3", "r4")
    df_VV_init_2 = df_tetra_order.select("r2", "r1", "r3", "r4")
    df_VV_init_3 = df_tetra_order.select("r3", "r1", "r2", "r4")
    df_VV_init_4 = df_tetra_order.select("r4", "r1", "r2", "r3")
    
    df_VV_union1234 = df_VV_init_1.union(df_VV_init_2).union(df_VV_init_3).union(df_VV_init_4)
    
    df_VV_gp = df_VV_union1234.groupBy('r1').agg(collect_set('r2').alias('multi_r2_set'),collect_set('r3').alias('multi_r3_set'),collect_set('r4').alias('multi_r4_set'))
    
    return df_VV_gp

In [6]:
df_VV_gp = get_VV(df_tetra_order)

df_VV_gp.printSchema()

root
 |-- r1: integer (nullable = true)
 |-- multi_r2_set: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- multi_r3_set: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- multi_r4_set: array (nullable = true)
 |    |-- element: integer (containsNull = true)



In [7]:
# obtain VV relation
def get_multi_pt_order(pt_list1, pt_list2, pt_list3):
# get_multi_pt_index is used to obtain the adjacent vertexes index, including the vertex itself
# pt_list1: partial adjacent vertex indexes of join result 1
# pt_list2: partial adjacent vertex indexes of join result 2
# pt_list3: partial adjacent vertex indexes of join result 3
    pt_init = pt_list1 + pt_list2 + pt_list3
    pt_set = set(pt_init)
    
    pt_list = sorted(pt_set) # sorted(pt_set, reverse=False), False in ascending order while True in descending order
    
    return pt_list # directly return a list, e.g., [0, 1, 5, 6], the length of returned column list will be 12, incluing "[" and empty char

get_multi_pt_order_udf = udf(get_multi_pt_order, ArrayType(IntegerType()))

In [8]:
df_VV = df_VV_gp.withColumn("VV", get_multi_pt_order_udf(df_VV_gp.multi_r2_set, df_VV_gp.multi_r3_set, df_VV_gp.multi_r4_set)).drop('multi_r2_set', 'multi_r3_set', 'multi_r4_set')

df_VV = df_VV.withColumnRenamed('r1', 'Ver')

df_VV.printSchema()

root
 |-- Ver: integer (nullable = true)
 |-- VV: array (nullable = true)
 |    |-- element: integer (containsNull = true)



In [9]:
t0 = time.time()

num_row = df_VV.count()

t1 = time.time()
print("Time cost:", t1 - t0)
print("number of rows:", num_row)

Time cost: 630.730774641037
number of rows: 1792989718


### Method 2: local method (get VV from VT)

##### load the pre-computed VT relation

In [4]:
# read a parquet file from hdfs
file_VT = directory + '/' + tin_filename + '_VT.parquet'

df_VT = spark.read.parquet(file_VT)
df_VT.printSchema()

root
 |-- Ver: integer (nullable = true)
 |-- VT: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)



In [6]:
t0 = time.time()

num_row = df_VT.count()

t1 = time.time()
print("Time cost:", t1 - t0)
# print("number of rows:", num_row)

Time cost: 15.129749536514282


In [7]:
# function to get VV from VT
def get_VV_from_VT(r1, multi_tetra_list):
# get_multi_pt_index is used to obtain the adjacent vertexes index, including the vertex itself

    pt_set = set()
    if len(multi_tetra_list) > 0:
        for tetra in multi_tetra_list:
            pt_set.update(tetra)
    
    pt_set.remove(r1)
    pt_list = sorted(pt_set) # sorted(pt_set, reverse=False), False in ascending order while True in descending order
    
    return pt_list # directly return a list, e.g., [0, 1, 5, 6], the length of returned column list will be 12, incluing "[" and empty char

get_VV_from_VT_udf = udf(get_VV_from_VT, ArrayType(IntegerType()))

In [8]:
df_VV = df_VT.withColumn("VV", get_VV_from_VT_udf(df_VT.Ver, df_VT.VT)).drop('VT')
# df_VV = df_VV.withColumnRenamed('r1', 'Ver')

df_VV.printSchema()

root
 |-- Ver: integer (nullable = true)
 |-- VV: array (nullable = true)
 |    |-- element: integer (containsNull = true)



In [9]:
t0 = time.time()

num_row = df_VV.count()
# df_VV.show()

t1 = time.time()
print("Time cost:", t1 - t0)
# print("number of rows:", num_row)

Time cost: 3.841904878616333


In [9]:
t0 = time.time()

file_VV = directory + '/' + tin_filename + '_VV.parquet'

df_VV.write.parquet(file_VV)

t1 = time.time()
print("time cost:", t1-t0)

time cost: 137.3480360507965


# obtain TT relation

### Method 1: pure global

In [5]:
# function to get TF relation
def get_TF_from_T(df_tetra_order):
    df_TF = df_tetra_order.withColumn("tetra", sort_array(F.array("r1", "r2", "r3", "r4"), False)).withColumn("f1", sort_array(F.array("r1", "r2", "r3"), False)).withColumn("f2", sort_array(F.array("r1", "r2", "r4"), False)).withColumn("f3", sort_array(F.array("r1", "r3", "r4"), False)).withColumn("f4", sort_array(F.array("r2", "r3", "r4"), False))
    df_TF = df_TF.select('tetra', 'f1', 'f2', 'f3', 'f4')
    
    return df_TF

In [6]:
# df_TF.persist(StorageLevel.MEMORY_ONLY)

df_TF = get_TF_from_T(df_tetra_order)

df_TF.printSchema()

root
 |-- tetra: array (nullable = false)
 |    |-- element: integer (containsNull = true)
 |-- f1: array (nullable = false)
 |    |-- element: integer (containsNull = true)
 |-- f2: array (nullable = false)
 |    |-- element: integer (containsNull = true)
 |-- f3: array (nullable = false)
 |    |-- element: integer (containsNull = true)
 |-- f4: array (nullable = false)
 |    |-- element: integer (containsNull = true)



In [7]:
df_FT_1 = df_TF.select('f1', 'tetra')
df_FT_2 = df_TF.select('f2', 'tetra')
df_FT_3 = df_TF.select('f3', 'tetra')
df_FT_4 = df_TF.select('f4', 'tetra')

df_FT_1234 = df_FT_1.union(df_FT_2).union(df_FT_3).union(df_FT_4)

df_FT = df_FT_1234.groupBy('f1').agg(collect_set('tetra').alias('FT'))
df_FT = df_FT.withColumnRenamed("f1", "Face")
df_FT.printSchema()

root
 |-- Face: array (nullable = false)
 |    |-- element: integer (containsNull = true)
 |-- FT: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)



In [8]:
# for boundary tetrahedron, its coboundary tetrahedron could be None
df_TT_init = df_FT.select(df_FT.FT[0].alias('t1'), df_FT.FT[1].alias('t2'))

df_TT_1 = df_TT_init.select('t1', 't2')
df_TT_2 = df_TT_init.select('t2', 't1')

df_TT_gp = df_TT_1.union(df_TT_2)
df_TT_gp = df_TT_gp.groupby('t1').agg(collect_set('t2').alias('multi_t2')).na.drop(subset=["t1"])
                                      
df_TT = df_TT_gp.withColumnRenamed("t1", "Tetra").withColumnRenamed("multi_t2", "TT")
df_TT.printSchema()

root
 |-- Tetra: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- TT: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)



In [9]:
t0 = time.time()

num_row = df_TT.count()

t1 = time.time()
print("Time cost:", t1 - t0)
print("number of rows:", num_row)

Time cost: 3963.778732061386
number of rows: 9715202459


### Method 2: symmetric global (get TT from TF and FT)

##### load the pre-computed TF relation

In [4]:
# read a parquet file from hdfs
file_TF = directory + '/' + tin_filename + '_TF.parquet'

df_TF = spark.read.parquet(file_TF)
df_TF.printSchema()

root
 |-- tetra: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- f1: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- f2: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- f3: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- f4: array (nullable = true)
 |    |-- element: integer (containsNull = true)



In [5]:
t0 = time.time()

num_row = df_TF.count()

t1 = time.time()
print("Time cost:", t1 - t0)
print("number of rows:", num_row)

Time cost: 21.21110224723816
number of rows: 9715202480


##### load the pre-computed FT relation

In [6]:
# read a parquet file from hdfs
file_FT = directory + '/' + tin_filename + '_FT.parquet'

df_FT = spark.read.parquet(file_FT)
df_FT.printSchema()

root
 |-- Face: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- FT: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)



In [7]:
t0 = time.time()

num_row = df_FT.count()

t1 = time.time()
print("Time cost:", t1 - t0)
print("number of rows:", num_row)

Time cost: 10.226840734481812
number of rows: 19434269798


In [8]:
df_TT_init_1 = df_TF.join(df_FT, df_TF.f1==df_FT.Face).select('tetra', col('FT').alias('TT_1'), 'f2', 'f3', 'f4')
df_TT_init_2 = df_TT_init_1.join(df_FT, df_TT_init_1.f2==df_FT.Face).select('tetra', 'TT_1', col('FT').alias('TT_2'), 'f3', 'f4')
df_TT_init_3 = df_TT_init_2.join(df_FT, df_TT_init_2.f3==df_FT.Face).select('tetra', 'TT_1', 'TT_2', col('FT').alias('TT_3'), 'f4')
df_TT_init_4 = df_TT_init_3.join(df_FT, df_TT_init_3.f4==df_FT.Face).select('tetra', 'TT_1', 'TT_2', 'TT_3', col('FT').alias('TT_4'))

df_TT_init_4.printSchema()

root
 |-- tetra: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- TT_1: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)
 |-- TT_2: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)
 |-- TT_3: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)
 |-- TT_4: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)



In [9]:
def get_TT_from_part_TT(tetra, TT_1, TT_2, TT_3, TT_4):
    TT = set()
    for t in TT_1:
        TT.add(tuple(t))
        
    for t in TT_2:
        TT.add(tuple(t))
        
    for t in TT_3:
        TT.add(tuple(t))
        
    for t in TT_4:
        TT.add(tuple(t))
        
    TT.remove(tuple(tetra))
    TT_list = list(TT)
    
    return TT_list

get_TT_from_part_TT_udf = udf(get_TT_from_part_TT, ArrayType(ArrayType(IntegerType())))

In [10]:
df_TT = df_TT_init_4.withColumn('TT', get_TT_from_part_TT_udf(df_TT_init_4.tetra, df_TT_init_4.TT_1, df_TT_init_4.TT_2, df_TT_init_4.TT_3, df_TT_init_4.TT_4))
df_TT = df_TT.select(col('tetra').alias('Tetra'), 'TT')
df_TT.printSchema()

root
 |-- Tetra: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- TT: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)



In [11]:
t0 = time.time()

num_row = df_TT.count()

t1 = time.time()
print("Time cost:", t1 - t0)
print("number of rows:", num_row)

Time cost: 1244.881771326065
number of rows: 9715202480
